In [ ]:
# Import required modules
import sys
sys.path.insert(0, '..')

from src.graph_builder import GraphBuilder
from src.workflow import app, AgentState
from src.models import Clause, Entity, Risk, RiskSeverity
from config.settings import settings

print(f"Neo4j URI: {settings.NEO4J_URI}")
print(f"Production mode: {settings.is_production}")

## 1. Connect to Neo4j and Explore

In [ ]:
# Test database connection
with GraphBuilder() as gb:
    stats = gb.get_graph_stats()
    print("Graph Statistics:")
    for label, count in stats.items():
        print(f"  {label}: {count} nodes")

## 2. Load and Analyze a Contract

In [ ]:
# Load sample contract
with open('../data/sample_contract.txt', 'r') as f:
    contract_text = f.read()

print(f"Loaded {len(contract_text):,} characters")
print("\n--- Preview ---")
print(contract_text[:500])

In [ ]:
# Run the analysis workflow
initial_state: AgentState = {
    "raw_text": contract_text,
    "extracted_clauses": [],
    "extracted_entities": [],
    "extracted_relationships": [],
    "compliance_report": ""
}

result = app.invoke(initial_state)

In [ ]:
# Display the compliance report
from IPython.display import Markdown
Markdown(result["compliance_report"])

## 3. Query the Knowledge Graph

In [ ]:
# Find all contradictions in the graph
with GraphBuilder() as gb:
    contradictions = gb.get_contradictions()
    
print(f"Found {len(contradictions)} contradictions:")
for c in contradictions:
    print(f"\n  Clause {c['clause1_id']} <-> Clause {c['clause2_id']}")
    print(f"  Reason: {c.get('contradiction_reason', 'N/A')}")

## 4. Custom Cypher Queries

You can run custom Cypher queries to explore the graph.

In [ ]:
# Custom query example: Find all relationships
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    settings.NEO4J_URI,
    auth=(settings.NEO4J_USER, settings.NEO4J_PASSWORD)
)

with driver.session() as session:
    result = session.run("""
        MATCH (a)-[r]->(b)
        RETURN labels(a)[0] as from_type, 
               type(r) as relationship, 
               labels(b)[0] as to_type,
               count(*) as count
        ORDER BY count DESC
    """)
    
    print("Relationship Summary:")
    for record in result:
        print(f"  ({record['from_type']})-[{record['relationship']}]->({record['to_type']}): {record['count']}")

driver.close()

## 5. Working with Models

In [ ]:
# Create typed model instances
from src.models import ContractAnalysis, Clause, Risk, RiskSeverity

# Build a structured analysis result
analysis = ContractAnalysis(
    contract_id="demo-contract-001",
    clauses=[
        Clause(id="1", topic="Indemnification", text="...")
    ],
    risks=[
        Risk(
            id="risk-1",
            severity=RiskSeverity.CRITICAL,
            description="Contradicting liability terms",
            clause_ids=["1", "2"]
        )
    ]
)

print(f"Contract: {analysis.contract_id}")
print(f"Critical Risks: {analysis.critical_risk_count}")
print(f"Has Contradictions: {analysis.has_contradictions}")